# Pckage List

In [4]:
import numpy as np, pandas as pd, warnings, os, openai, json
from tqdm.auto import tqdm
from openai import OpenAI
warnings.filterwarnings('ignore')
import requests
from typing import List, Tuple, Union
from langchain_ollama.embeddings import OllamaEmbeddings
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from haversine import haversine, Unit

# Data Load

In [5]:
df = pd.read_csv("../data/prep2.csv", encoding='cp949')

# Function

In [12]:
class OllamaSentenceTransformer():
    def __init__(
            self,
            *args,
            **kargs,
            ) -> None:
                # self.base_url = kargs.get()
                self.model = kargs.get("model","bge-m3")
                self.embedding_model = OllamaEmbeddings(
                            model = self.model,
                            # base_url = self.base_rul,
                        )
                        
    
    def encode(
            self,
            documents:Union[str, List[str], np.ndarray],
            *args,
            **kargs,
        )-> np.ndarray:
        if isinstance(documents, str):
            document_embeddings = self.embedding_model.embed_query(documents)
            return np.array(document_embeddings)
        
        if isinstance(documents, np.ndarray):
            documents = documents.tolist()
        
        document_embeddings = [self.embedding_model.embed_query(s) for s in documents]
        return np.array(document_embeddings)
        
sentence_transformer = OllamaSentenceTransformer()

## 위치 관련 정보 제공

In [6]:
def get_lat_lon(location_name):
    url = 'https://nominatim.openstreetmap.org/search'
    params = {
        'q': location_name,
        'format': 'json'
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (compatible; ChatGPT-Example/1.0; +http://yourdomain.com)'
    }

    response = requests.get(url, params=params, headers=headers)

    # 응답 확인
    if response.status_code != 200:
        print(f"요청 실패: {response.status_code}")
        print("응답 내용:", response.text)
        return None

    try:
        data = response.json()
        if data:
            lat = data[0]['lat']
            lon = data[0]['lon']
            return float(lat), float(lon)
        else:
            print("위치 정보를 찾을 수 없습니다.")
            return None
    except ValueError as e:
        print("JSON 파싱 오류:", e)
        print("응답 내용:", response.text)
        return None

# 테스트
location = '진천군'
result = get_lat_lon(location)
print(f"{location} 위도/경도: {result}")


진천군 위도/경도: (36.855461, 127.4353927)


## API 키 설정

In [7]:
key = open('../../../api_key.txt','r')
api_key = key.read()
openai.api_key = api_key

# 전체 툴 인코딩 값

In [8]:
with open('../data/encoded_tool_name2.pkl', "rb") as f:
    all_tool_encoded_array = pickle.load(f)

# Input Content

In [28]:
input_content = {
    'tool_name': '전동드릴',
    'location': '강남역',
    'job_content': '전동드릴을 사용해서 나무를 고정하고 싶어',    
}

In [29]:
encoded_tool_name = sentence_transformer.encode(input_content['tool_name'])
location_name = input_content['location']
spot_location = get_lat_lon(location_name)



In [17]:
def compare_cosim(all_asset_encoded_val:np.array, asset_info:np.array) -> float:
    """
    두 임베딩 벡터의 코사인 유사도를 계산합니다.
    
    Args:
        all_asset_encoded_val (np.array): 첫 번째 임베딩 벡터
        asset_info (np.array): 두 번째 임베딩 벡터
    
    Returns:
        float: 코사인 유사도
    """
    # return np.dot(all_asset_encoded_val, asset_info) / (np.linalg.norm(all_asset_encoded_val) * np.linalg.norm(asset_info))
    
    cos_sim = cosine_similarity(all_asset_encoded_val, [asset_info])
    return cos_sim

In [21]:
df['tool_sim_result'] = compare_cosim(all_tool_encoded_array, encoded_tool_name)

df_s = df[df['tool_sim_result']>0.9]
df_s['Distance'] = df_s[["위도","경도"]].apply(lambda x : haversine((spot_location[0], spot_location[1]), (x['위도'], x['경도']), unit=Unit.KILOMETERS), axis=1)
df_s.sort_values(by='Distance', ascending=True, inplace=True)

In [23]:
df_s.head(3)

,구분자(PK),공구 대분류 코드,공구 대분류,공구 중분류 코드,공구 중분류,공구 이름,과금기준,수량,장소 PK,대여장소명,...,경도,전화번호,평일오픈시간,평일클로즈시간,생성일시,요금,cluster,공구 중분류 new,tool_sim_result,Distance
10414,6861,2,전동공구,21,전동드릴,전동드릴,1일,3,243,서초1동반딧불센터,...,127.021720,02-521-0364,1000.0,1800.0,2018-01-23 17:33:00,10000원,354,전동드릴,1.0,1.255399
10423,6818,2,전동공구,21,전동드릴,전동드릴,1일,1,238,반포반딧불센터,...,127.018460,02-516-7887,1000.0,1800.0,2018-01-23 17:33:00,10000원,354,전동드릴,1.0,1.985759
10306,15925,2,전동공구,21,전동드릴,전동드릴,없음,1,13360,잠원동주민센터,...,127.014077,02-2155-7543,900.0,1800.0,2023-08-02 07:49:01,무료,354,전동드릴,1.0,2.681247


In [25]:
client = OpenAI(api_key=api_key)
def invoke(question, info = None, model="gpt-4o", temperature=0.7):
    response = client.chat.completions.create(
        model=model,
        messages = [
                    {"role": "system", 
                        "content": 
                            
                        
                            f"""너는 사람들의 질문에 친절히 답해주는 도우미야.
                        너는 서울시에서 운영하는 대여 공구 찾기 정보 시스템이야. 
                        관련 정보는 {info}와 같아. 
                        이정보를 통해서 사용자의 답변에 친절해 답해줘.
                        """ if info != None else
                            f"""너는 사람들의 질문에 친절히 답해주는 도우미야.
                        너는 서울시에서 운영하는 대여 공구 찾기 정보 시스템이야. 
                        사용자의 답변에 친절해 답해줘.
                        """
                        },
                    {"role": "user", 
                    "content": question}
                ],
        temperature=temperature
    )
    return response.choices[0].message.content

In [30]:
input_content[tool_name]

NameError: name 'tool_name' is not defined

In [31]:
question = f"나는 {input_content['tool_name']}을(를) 빌리고 싶어. 그리고 이 도구를 이용해서 하려는 작업은 '{input_content['job_content']}' 이야. 해당 작업을 하면서 같이 빌리면 좋은 공구도 함께 알려줘"

In [ ]:
import pickle
with open('../data/encoded_tool_name2.pkl', 'wb') as f:
    pickle.dump(embeddings, f)

In [32]:
answer = invoke(question)
print("💬 GPT 응답:", answer)

💬 GPT 응답: 전동드릴을 사용해서 나무를 고정하려는 작업을 계획하고 계시군요! 전동드릴은 매우 유용한 도구로, 나무를 고정하는 데 적합한 선택입니다. 이 작업을 하면서 함께 빌리면 좋은 공구는 다음과 같습니다:

1. **드릴 비트 세트**: 나무 작업에 맞는 다양한 크기의 드릴 비트가 포함된 세트가 필요할 수 있습니다. 이는 나사 구멍을 뚫거나 나사못을 고정할 때 유용합니다.

2. **나사못 세트**: 나무를 고정할 때 적합한 크기와 길이의 나사못이 필요합니다. 다양한 사이즈의 나사못이 포함된 세트를 준비하면 좋습니다.

3. **클램프**: 나무를 고정하는 동안 안정적으로 잡아줄 클램프가 있으면 작업이 훨씬 수월합니다. 특히, 두 개 이상의 나무를 함께 고정할 때 유용합니다.

4. **각도자**: 나무를 일정한 각도로 고정하거나 자를 때 각도자는 정확한 작업을 도와줍니다.

5. **수평계**: 나무가 수평으로 고정되었는지 확인하는 데 사용됩니다.

6. **작업용 장갑**: 손을 보호하고 작업의 안전성을 높이는 데 도움이 됩니다.

서울시에서 운영하는 공구 대여 서비스를 통해 필요한 공구를 대여할 수 있습니다. 대여 가능 여부와 위치를 확인하시기 위해 가까운 주민센터나 구청에 문의하시거나, 서울시 공구 대여 서비스 웹사이트를 방문해 보세요. 즐거운 작업 되시길 바랍니다!


In [ ]:
def invoke(question: str):
    prompt = f"""
    너는 도우미야 사용자의 물음에 친절히 답해줘야해.
    """
    response = client.responses.create(
        model='gpt-4.1',
        messages=[
                {"role": "system", "content": "당신은 완벽한 도우미입니다."},
                {"role": "user", "content": prompt}
            ],
        input = question
        )
    return response